In [4]:
import numpy as np
TRAIN_FILE = 'fashion_mnist/train.csv'
TEST_FILE = 'fashion_mnist/test.csv'
VAL_FILE = 'fashion_mnist/val.csv'

In [5]:
data = np.genfromtxt(TRAIN_FILE,delimiter=',')

In [6]:
data.shape

(22500, 785)

In [7]:
classes = [0,1] # Binary classification 
newdata = np.asarray([r for r in data if r[-1] in classes])

In [8]:
# change classes to {-1,1}
# change values to [0,1] from [0,255]
X = newdata[:,0:784] / 255.0
Y = 2 * np.asarray(newdata[:,-1]) -1 
Y = np.asarray([Y])

In [9]:
Z = Y.T * X

In [10]:
## M x M array
P = np.matmul(Z, Z.T)

In [11]:
# column vector with all -1
Q = -1 * np.ones((len(P),1)).astype(float)

In [12]:
G = -1 * np.eye(len(P)).astype(float)

In [13]:
h = np.zeros((len(P),1)).astype(float)

In [14]:
A = Y
b = [[0]]

In [15]:
import cvxopt as cv


In [17]:
A_ = cv.matrix(A)
P_ = cv.matrix(P)
Q_ = cv.matrix(Q)
G_ = cv.matrix(G)
h_ = cv.matrix(h)
b_ = cv.matrix(b,tc='d')

In [18]:
sol = cv.solvers.qp(P_,Q_,G_,h_)

     pcost       dcost       gap    pres   dres
 0: -3.5671e+02 -1.0801e+03  2e+04  1e+02  3e+00
 1: -5.6674e+02 -9.9101e+02  1e+04  6e+01  1e+00
 2: -3.1508e+02 -3.0559e+02  4e+03  1e+01  4e-01
 3: -1.1899e+02 -1.1201e+02  1e+03  4e+00  9e-02
 4: -5.4013e+01 -6.6768e+01  4e+02  1e+00  3e-02
 5: -3.4063e+01 -4.8975e+01  2e+02  5e-01  1e-02
 6: -2.5914e+01 -4.0621e+01  6e+01  1e-01  3e-03
 7: -2.7401e+01 -3.6223e+01  3e+01  5e-02  1e-03
 8: -2.7638e+01 -3.5074e+01  8e+00  3e-03  8e-05
 9: -3.0988e+01 -3.4098e+01  3e+00  1e-03  3e-05
10: -3.2654e+01 -3.3708e+01  1e+00  1e-15  9e-13
11: -3.3500e+01 -3.3615e+01  1e-01  1e-15  9e-13
12: -3.3604e+01 -3.3606e+01  2e-03  1e-15  1e-12
13: -3.3606e+01 -3.3606e+01  3e-05  1e-15  1e-12
Optimal solution found.


In [19]:
alph = np.array(sol['x'])

In [20]:
fw = alph * Y.T

In [21]:
w = np.matmul(fw.T,X)

In [22]:
R = np.matmul(w,X.T)

In [23]:
maxv = 0.0
minv = 100000000.0
for i in range(0,len(R[0])):
    if Y[0][i] == 1:
        if(R[0][i] < minv):
            minv = R[0][i]
    else:
        if(R[0][i]>maxv):
            maxv = R[0][i]
b = -(maxv+minv)/2.0

In [24]:
tdata = np.genfromtxt(TEST_FILE,delimiter=',')

In [25]:
tY = tdata[:,-1]

In [26]:
tX = tdata[:,0:784]

tR = np.matmul(w,tX.T)

In [27]:
tot = 0
corr = 0
for i in range(0,len(tR[0])):
    if(tY[i] not in classes):
        continue;
    tot += 1;
    if((tR[0][i]>0 and tY[i]==1) or (tR[0][i]<0 and tY[i]==0)):
        corr +=1;

In [28]:
print(float(corr)/float(tot))

0.977
